# Data Processing - NSRDB dataset

Author: Huiting Song

In [1]:
# Import packages
import pandas as pd
import sklearn
import numpy as np
import nltk
import json
import matplotlib.pyplot as plt
import seaborn as sns

## Datasets from NSRDB 

The datasets from this file are all from NSRDB -- The National Solar Radiation Database.

### Arlington

In [3]:
# Read cvs files of Arlington and from 2019 to 2022
Arlington2019 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Arlington_NSRDB_2019.csv")
Arlington2020 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Arlington_NSRDB_2020.csv")
Arlington2021 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Arlington_NSRDB_2021.csv")
Arlington2022 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Arlington_NSRDB_2022.csv")

# Appending four dataframes
Arlington = Arlington2019.append([Arlington2020, Arlington2021, Arlington2022],ignore_index=True)

# Displaying the dataframe 
pd.options.display.max_rows = 10 
pd.options.display.max_columns = 25
display(Arlington) # totally 35040 observations and 23 variables

# save the appended dataframe to the RAW data folder
Arlington.to_csv('/Users/crystal/Desktop/ANLY5550/Data/Raw/NSRDB_Arlington.csv', index=False)

/var/folders/44/9f70kzrs4hd1xjzn2ldh0hqh0000gn/T/ipykernel_32947/4001599879.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Arlington = Arlington2019.append([Arlington2020, Arlington2021, Arlington2022],ignore_index=True)


,Year,Month,Day,Hour,Minute,DHI,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,DNI,Fill Flag,GHI,Ozone,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed
0,2019,1,1,0,0,0,11.8,0,0,0,6,11.7,0,0,0,0.224,99.21,163.93,0.07,1003,3.9,206,4.5
1,2019,1,1,1,0,0,12.6,0,0,0,6,12.6,0,0,0,0.224,99.72,161.08,0.07,1001,3.8,206,4.7
2,2019,1,1,2,0,0,13.2,0,0,0,7,13.2,0,0,0,0.226,99.96,152.05,0.07,1001,3.3,217,4.7
3,2019,1,1,3,0,0,13.6,0,0,0,7,13.6,0,0,0,0.227,99.85,141.01,0.07,1001,2.7,228,4.5
4,2019,1,1,4,0,0,13.9,0,0,0,8,13.9,0,0,0,0.229,99.95,129.42,0.07,1001,2.2,234,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,2022,12,31,19,0,0,10.5,0,0,0,8,9.9,0,0,0,0.296,95.87,113.30,0.07,1006,2.6,190,1.4
35036,2022,12,31,20,0,0,10.5,0,0,0,8,10.0,0,0,0,0.296,96.52,124.86,0.07,1006,2.7,200,1.3
35037,2022,12,31,21,0,0,10.4,0,0,0,4,10.1,0,0,0,0.294,97.80,136.51,0.07,1006,2.8,202,1.2
35038,2022,12,31,22,0,0,10.3,0,0,0,7,10.1,0,0,0,0.293,98.62,147.86,0.07,1005,2.9,185,1.1


### Richmond

In [4]:
# Read cvs files of Richmond and from 2019 to 2022
Richmond2019 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Richmond_NSRDB_2019.csv")
Richmond2020 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Richmond_NSRDB_2020.csv")
Richmond2021 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Richmond_NSRDB_2021.csv")
Richmond2022 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Richmond_NSRDB_2022.csv")

# Appending four dataframes
Richmond = Richmond2019.append([Richmond2020, Richmond2021, Richmond2022],ignore_index=True)

# Displaying the dataframe 
pd.options.display.max_rows = 10 
pd.options.display.max_columns = 25
display(Richmond) # totally 35040 observations and 23 variables

# save the appended dataframe to the RAW data folder
Arlington.to_csv('/Users/crystal/Desktop/ANLY5550/Data/Raw/NSRDB_Richmond.csv', index=False)

/var/folders/44/9f70kzrs4hd1xjzn2ldh0hqh0000gn/T/ipykernel_32947/4061906815.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Richmond = Richmond2019.append([Richmond2020, Richmond2021, Richmond2022],ignore_index=True)


,Year,Month,Day,Hour,Minute,DHI,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,DNI,Fill Flag,GHI,Ozone,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed
0,2019,1,1,0,0,0,13.6,0,0,0,6,13.6,0,0,0,0.228,100.00,165.22,0.12,1008,3.9,193,0.7
1,2019,1,1,1,0,0,13.9,0,0,0,7,13.9,0,0,0,0.227,100.00,162.32,0.12,1007,3.9,195,0.8
2,2019,1,1,2,0,0,14.3,0,0,0,6,14.3,0,0,0,0.226,100.00,152.88,0.12,1007,4.0,201,0.9
3,2019,1,1,3,0,0,14.7,0,0,0,7,14.7,0,0,0,0.224,100.00,141.52,0.12,1006,4.0,209,0.9
4,2019,1,1,4,0,0,14.8,0,0,0,7,14.8,0,0,0,0.224,100.00,129.69,0.12,1006,3.5,213,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,2022,12,31,19,0,0,12.5,0,0,0,7,10.4,0,0,0,0.309,87.18,112.76,0.12,1009,2.3,176,0.6
35036,2022,12,31,20,0,0,11.4,0,0,0,4,10.5,0,0,0,0.308,94.29,124.52,0.12,1009,2.4,175,0.6
35037,2022,12,31,21,0,0,11.5,0,0,0,7,11.0,0,0,0,0.304,96.84,136.39,0.12,1008,2.8,177,0.6
35038,2022,12,31,22,0,0,12.3,0,0,0,3,11.9,0,0,0,0.302,97.64,148.04,0.12,1008,3.0,195,0.7


### Shenandoah

In [5]:
# Read cvs files of Richmond and from 2019 to 2022
Shenandoah2019 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Shenandoah_NSRDB_2019.csv")
Shenandoah2020 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Shenandoah_NSRDB_2020.csv")
Shenandoah2021 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Shenandoah_NSRDB_2021.csv")
Shenandoah2022 = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/Shenandoah_NSRDB_2022.csv")

# Appending four dataframes
Shenandoah = Shenandoah2019.append([Shenandoah2020, Shenandoah2021, Shenandoah2022],ignore_index=True)

# Displaying the dataframe 
pd.options.display.max_rows = 10 
pd.options.display.max_columns = 25
display(Shenandoah) # totally 35040 observations and 23 variables

# save the appended dataframe to the RAW data folder
Arlington.to_csv('/Users/crystal/Desktop/ANLY5550/Data/Raw/NSRDB_Shenandoah.csv', index=False)

/var/folders/44/9f70kzrs4hd1xjzn2ldh0hqh0000gn/T/ipykernel_32947/674216614.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Shenandoah = Shenandoah2019.append([Shenandoah2020, Shenandoah2021, Shenandoah2022],ignore_index=True)


,Year,Month,Day,Hour,Minute,DHI,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,DNI,Fill Flag,GHI,Ozone,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed
0,2019,1,1,0,0,0,11.9,0,0,0,6,11.7,0,0,0,0.222,98.84,164.08,0.12,971,3.6,211,1.2
1,2019,1,1,1,0,0,12.1,0,0,0,7,12.0,0,0,0,0.224,99.14,162.11,0.12,970,3.2,217,1.2
2,2019,1,1,2,0,0,11.9,0,0,0,7,11.8,0,0,0,0.225,99.62,153.31,0.12,971,2.5,226,1.1
3,2019,1,1,3,0,0,11.7,0,0,0,1,11.6,0,0,0,0.227,99.62,142.27,0.12,971,2.0,237,1.1
4,2019,1,1,4,0,0,11.7,0,0,0,0,11.3,0,0,0,0.229,97.53,130.62,0.12,971,1.6,248,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,2022,12,31,19,0,0,11.7,0,0,0,4,9.2,0,0,0,0.293,84.89,112.04,0.12,974,2.6,192,0.7
35036,2022,12,31,20,0,0,11.3,0,0,0,7,9.4,0,0,0,0.291,88.16,123.64,0.12,973,2.8,195,0.7
35037,2022,12,31,21,0,0,11.2,0,0,0,8,9.5,0,0,0,0.290,89.45,135.36,0.12,973,2.8,197,0.7
35038,2022,12,31,22,0,0,10.9,0,0,0,7,9.5,0,0,0,0.288,91.14,146.85,0.12,973,2.8,211,0.8
